In [3]:
# --- 0) INSTALLS (uncomment if needed) ---------------------------------------
# %pip install pytesseract pdf2image pillow pandas openpyxl

# --- 1) IMPORTS ---------------------------------------------------------------
import re
import io
from pathlib import Path
from typing import List, Tuple, Optional

import pandas as pd
from PIL import Image, ImageOps, ImageFilter
import pytesseract
from pdf2image import convert_from_path

# --- 2) CONFIG ----------------------------------------------------------------
INPUT_PDF   = Path("/users/coop/desktop/AP Recon/mayer_aug25.pdf")
OUTPUT_XLSX = Path("/users/coop/desktop/AP Recon/mayer_aug25.xlsx")
DPI         = 300  # rasterization DPI for OCR, 300–400 usually good

# If Tesseract is not on PATH (Windows), set it explicitly, e.g.:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Common invoice formats. Adjust as needed.
INVOICE_PATTERNS = [
    r"\b\d{6,10}\b",                # pure digits length 6–10
    r"\b[A-Z]{2}\d{6,}\b",          # e.g., AB123456
    r"\b[A-Z0-9]{3,}-[A-Z0-9]{2,}\b" # general alphanum with dash
]
# Optional helpers: amounts and dates for context only
AMOUNT_PATTERN = r"(?:USD|US\$|\$)\s?(\d{1,3}(?:,\d{3})*(?:\.\d{2})|\d+\.\d{2})"
DATE_PATTERN   = r"\b(20\d{2}[-/\.](?:0?[1-9]|1[0-2])[-/\.](?:0?[1-9]|[12]\d|3[01])|\b(?:0?[1-9]|1[0-2])[-/\.](?:0?[1-9]|[12]\d|3[01])[-/\.]20\d{2})\b"

# --- 3) HELPERS ---------------------------------------------------------------
def preprocess(img: Image.Image) -> Image.Image:
    """Light cleanup to help OCR."""
    g = ImageOps.grayscale(img)
    g = ImageOps.autocontrast(g)
    # mild sharpen can help small text
    g = g.filter(ImageFilter.UnsharpMask(radius=1.2, percent=150, threshold=8))
    return g

def ocr_page(img: Image.Image) -> str:
    cfg = "--oem 3 --psm 6"  # assume a block of text
    return pytesseract.image_to_string(img, config=cfg)

def find_invoice_hits(text: str, page_num: int) -> List[Tuple[str,int,str,Optional[str],Optional[str]]]:
    """
    Return tuples: (invoice_no, page, line_no, amount_guess, date_guess)
    """
    lines = text.splitlines()
    combined_re = re.compile("|".join(f"(?:{p})" for p in INVOICE_PATTERNS))
    amt_re  = re.compile(AMOUNT_PATTERN)
    date_re = re.compile(DATE_PATTERN)
    hits = []
    for i, line in enumerate(lines, start=1):
        if not line.strip():
            continue
        for m in combined_re.finditer(line):
            inv = m.group(0)
            # grab nearby context on the same line
            amt = None
            dt  = None
            am = amt_re.search(line)
            if am:
                amt = am.group(0)  # include symbol for clarity
            dm = date_re.search(line)
            if dm:
                dt = dm.group(0)
            hits.append((inv, page_num, i, amt, dt, line.strip()))
    return hits

# --- 4) MAIN ------------------------------------------------------------------
def extract_invoices_from_pdf(pdf_path: Path) -> pd.DataFrame:
    # Convert all pages to images
    pages = convert_from_path(str(pdf_path), dpi=DPI)
    rows = []
    for pnum, page in enumerate(pages, start=1):
        img = preprocess(page)
        txt = ocr_page(img)
        for inv, pg, ln, amt, dt, src in find_invoice_hits(txt, pnum):
            rows.append({
                "invoice_no": inv,
                "page": pg,
                "line": ln,
                "amount_hint": amt,
                "date_hint": dt,
                "source_line": src
            })
    df = pd.DataFrame(rows)
    if df.empty:
        return df
    # Heuristic de-duplication: keep first occurrence of each invoice_no
    df = df.sort_values(["invoice_no", "page", "line"]).drop_duplicates(subset=["invoice_no"], keep="first")
    # Optional: normalize invoice_no (remove commas/spaces)
    df["invoice_no_norm"] = df["invoice_no"].str.replace(r"[,\s]", "", regex=True)
    return df

df = extract_invoices_from_pdf(INPUT_PDF)

# --- 5) SAVE AND PREVIEW ------------------------------------------------------
if df.empty:
    print("No invoice-like strings found. Tune INVOICE_PATTERNS or DPI, or try different preprocessing.")
else:
    print(df.head(20).to_string(index=False))
    OUTPUT_XLSX.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as xw:
        df.to_excel(xw, sheet_name="invoices", index=False)
    print(f"\nWrote: {OUTPUT_XLSX}")

invoice_no  page  line amount_hint date_hint                                                                           source_line invoice_no_norm
   1386378     1    33        None      None     RI 34881032 07/18/25 34881032 RETURN -3,710.00 -3,710.00 0.00 45081201 1386378 co         1386378
   1394313     2    24        None      None     Rl 34938285 07/31/25 34938285 6280-71522 -134.69 -134.69 0.00 45166681 1394313 co         1394313
   1641792     2    21        None      None                Rl 34850814 07/11/25 34850814 6280-71433 267.43 267.43 0.00 1641792 SD         1641792
   1647607     1    37        None      None            RI 34918516 07/28/25 34918516 6300-70369 4,813.49 4,813.49 0.00 1647607 SD         1647607
  22319480     1    32        None      None                                                22319480 WHITEHEAD ELECTRIC-QTS OSGOOD        22319480
  22597352     1    35        None      None                                               22597352 WHITEHEAD ELECTRIC

In [ ]:
# WORKING MAYER

from pathlib import Path
import re
import unicodedata
import pandas as pd
from PIL import Image, ImageOps, ImageFilter
import pytesseract
from pdf2image import convert_from_path

PDF   = Path("/users/coop/desktop/AP Recon/mayer_aug25.pdf")
OUT = Path("/users/coop/desktop/AP Recon/mayer_aug25.xlsx")
DPI = 300

# If Tesseract is not on PATH (Windows):
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def preprocess(img: Image.Image) -> Image.Image:
    g = ImageOps.grayscale(img)
    g = ImageOps.autocontrast(g)
    return g.filter(ImageFilter.UnsharpMask(radius=1.2, percent=150, threshold=8))

def ocr_text(img: Image.Image) -> str:
    # preserve spacing so columns stay aligned
    cfg = "--oem 3 --psm 6 -c preserve_interword_spaces=1"
    return pytesseract.image_to_string(img, config=cfg)

def norm(s: str) -> str:
    # normalize weird dashes and digits from OCR
    s = unicodedata.normalize("NFKC", s)
    return s.replace("\u2212", "-").replace("–", "-").replace("—", "-")

# Pattern: RI/Rl  <inv>  <mm/dd/yy>  <orig_inv>  <PO or RETURN>  <orig_amt> ...
PAT = re.compile(
    r"""^\s*R[Il1]\s+                # RI or Rl or R1 (OCR variance)
        (?P<invoice>\d{5,})\s+       # invoice number
        (?P<date>\d{2}/\d{2}/\d{2})\s+
        (?P<orig_inv>\d{5,})\s+
        (?P<po>[A-Za-z0-9-]+)\s+
        (?P<orig_amt>[\(\)\-$\d,.\u2212–—-]+)   # up through original amount
    """,
    re.VERBOSE,
)

def parse_amount(s: str) -> float | None:
    t = norm(s).replace("$", "").replace(",", "").strip()
    if t.startswith("(") and t.endswith(")"):
        t = "-" + t[1:-1]
    m = re.search(r"-?\d+(?:\.\d+)?", t)
    return float(m.group(0)) if m else None

rows = []
for pnum, page in enumerate(convert_from_path(str(PDF), dpi=DPI), start=1):
    text = ocr_text(preprocess(page))
    for raw in text.splitlines():
        line = norm(raw).strip()
        m = PAT.match(line)
        if not m:
            continue
        rows.append(
            {
                "invoice_no": m.group("invoice"),
                "invoice_date": m.group("date"),
                "po": m.group("po"),
                "original_amount": parse_amount(m.group("orig_amt")),
                "source_line": line,   # keep for quick QA; drop later if unwanted
                "page": pnum,
            }
        )

df = pd.DataFrame(rows, columns=["invoice_no","invoice_date","po","original_amount","source_line","page"])
if df.empty:
    print("No matching statement rows found. Check the pattern or DPI.")
else:
    # Coerce date
    df["invoice_date"] = pd.to_datetime(df["invoice_date"], format="%m/%d/%y", errors="coerce")
    print(df.head(20).to_string(index=False))
    OUT.parent.mkdir(parents=True, exist_ok=True)
    df[["invoice_no","invoice_date","po","original_amount"]].to_excel(OUT, index=False)
    print(f"\nWrote: {OUT}")

invoice_no invoice_date         po  original_amount                                                                                                                                                                                                                    source_line  page
  34853334   2025-07-11 6460-71539          2867.86                                 RI        34853334      07/11/25         34853334       6460-71539                                     2,867.86                 2,867.86               0.00 45027409         25090009       so     1
  34907099   2025-07-24 6360-71497            27.96 RI          34907099        07/24/25           34907099        6360-71497                                                   27.96                           27.96                   0.00 45091983           25043679        so     1
  34919950   2025-07-28 6460-71578          2717.05              RI         34919950       07/28/25          34919950        6460-71578                      

In [9]:
# %pip install pytesseract pdf2image pillow pandas openpyxl

from pathlib import Path
import re, unicodedata
import pandas as pd
from PIL import Image, ImageOps, ImageFilter
from pdf2image import convert_from_path
import pytesseract

# ---------- CONFIG ----------
FOLDER   = Path("/users/coop/desktop/Recon/Graybar")
OUT_XLSX = Path("/users/coop/desktop/Recon/graybar.xlsx")
DPI = 300
# If Tesseract not on PATH (Windows), set:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ---------- OCR helpers ----------
def preprocess(img):
    g = ImageOps.grayscale(img)
    g = ImageOps.autocontrast(g)
    return g.filter(ImageFilter.UnsharpMask(radius=1.2, percent=150, threshold=8))

def ocr_page(img):
    cfg = "--oem 3 --psm 6 -c preserve_interword_spaces=1"
    return pytesseract.image_to_string(img, config=cfg)

def norm(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    return s.replace("\u2212", "-").replace("–", "-").replace("—", "-")

# ---------- Patterns per your spec ----------
DATE_INV_RE = re.compile(
    r"^\s*(?P<date>\d{2}/\d{2}/\d{4})\s+(?P<inv>(?:93|14)\d{8})\b(?P<tail>.*)$"
)
NUM_RE = re.compile(r"-?\(?\$?\d{1,3}(?:,\d{3})*(?:\.\d+)?\)?|-?\$?\d+(?:\.\d+)?")

def parse_amount(last_token: str) -> float | None:
    t = norm(last_token).replace("$", "").replace(",", "").strip()
    if t.startswith("(") and t.endswith(")"):
        t = "-" + t[1:-1]
    m = re.search(r"-?\d+(?:\.\d+)?", t)
    return float(m.group(0)) if m else None

# ---------- Main ----------
rows = []
pdfs = sorted(p for p in FOLDER.glob("*.pdf") if p.is_file())
if not pdfs:
    raise SystemExit(f"No PDFs found in {FOLDER}")

for pdf in pdfs:
    for pnum, page in enumerate(convert_from_path(str(pdf), dpi=DPI), start=1):
        text = ocr_page(preprocess(page))
        for raw in text.splitlines():
            line = norm(raw.strip())
            m = DATE_INV_RE.match(line)
            if not m:
                continue
            # last numeric on the line is the amount due
            nums = NUM_RE.findall(m.group("tail"))
            if not nums:
                continue
            amt = parse_amount(nums[-1])
            rows.append(
                {
                    "invoice_date": m.group("date"),
                    "invoice_no": m.group("inv"),
                    "amount_due": amt,
                }
            )

df = pd.DataFrame(rows, columns=["invoice_date","invoice_no","amount_due"])
if df.empty:
    print("No matching invoice lines found. Check OCR quality or patterns.")
else:
    df["invoice_date"] = pd.to_datetime(df["invoice_date"], format="%m/%d/%Y", errors="coerce")
    print(df.head(20).to_string(index=False))
    OUT_XLSX.parent.mkdir(parents=True, exist_ok=True)
    df.to_excel(OUT_XLSX, index=False)
    print(f"\nWrote: {OUT_XLSX}")

invoice_date invoice_no  amount_due
  2025-06-26 9342529590       73.43
  2025-07-14 9300212387    19287.48
  2025-07-18 9300270428      401.21
  2025-07-18 9300289514      882.12
  2025-07-21 9300308393      626.31
  2025-07-25 1400074737        1.93
  2025-07-25 1400074737       26.25
  2025-07-25 1400074737        7.17
  2025-07-25 1400074737        7.82
  2025-07-29 9300420117      617.81
  2025-08-06 9300531700      195.12
  2025-08-11 1400099768        2.67
  2025-08-11 1400099768        5.38
  2025-08-11 1400099768       15.44
  2025-08-11 1400099768       77.66
  2025-08-11 1400099768       17.60
  2025-08-12 9300603125       47.78
  2025-08-13 9300614498     1235.62
  2025-08-21 9300736456      195.12
  2025-08-21 9300736467      254.25

Wrote: /users/coop/desktop/Recon/graybar.xlsx


In [20]:
# pip install cloudscraper lxml bs4 pandas
import cloudscraper
from bs4 import BeautifulSoup, Comment
import pandas as pd

url = "https://www.pro-football-reference.com/boxscores/202509070buf.htm"
table_id = "home_snap_counts"  # or "vis_snap_counts"

scraper = cloudscraper.create_scraper(browser={"custom": "Chrome/126"})
html = scraper.get(url, timeout=30).text  # single web request

soup = BeautifulSoup(html, "html.parser")
container = soup.find(id=f"all_{table_id}")
if not container:
    raise RuntimeError(f"Wrapper all_{table_id} not found")

comment = next((n for n in container.descendants if isinstance(n, Comment) and "<table" in n), None)
if not comment:
    raise RuntimeError(f"Commented table {table_id} not found")

df = pd.read_html(str(comment))[0]
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(-1)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
print(df)

SSLError: HTTPSConnectionPool(host='www.pro-football-reference.com', port=443): Max retries exceeded with url: /boxscores/202509070buf.htm (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))

In [22]:
# pip install selenium webdriver-manager bs4 lxml pandas

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup, Comment
import pandas as pd

url = "https://www.pro-football-reference.com/boxscores/202509070atl.htm"
table_id = "home_snap_counts"  # use "vis_snap_counts" for away

# 1) Launch headless Chrome with a realistic UA
opts = Options()
opts.add_argument("--headless=new")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-gpu")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/126.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

try:
    # 2) Single navigation
    driver.get(url)

    # Optional: wait until the wrapper div for the hidden table is present
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, f"#all_{table_id}"))
    )

    html = driver.page_source
finally:
    driver.quit()

# 3) Parse commented table HTML and load into pandas
soup = BeautifulSoup(html, "html.parser")
container = soup.find(id=f"all_{table_id}")
if container is None:
    raise RuntimeError(f"Wrapper all_{table_id} not found")

comment = next((n for n in container.descendants if isinstance(n, Comment) and "<table" in n), None)
if comment is None:
    raise RuntimeError(f"Commented table '{table_id}' not found")

df = pd.read_html(str(comment))[0]
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(-1)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
print(df)

               Player Pos  Num   Pct  Num   Pct  Num  Pct
0    Elijah Wilkinson  OL   76  100%    0    0%    5  19%
1    Matthew Bergeron  OL   76  100%    0    0%    5  19%
2       Jake Matthews   T   76  100%    0    0%    5  19%
3     Chris Lindstrom   G   76  100%    0    0%    5  19%
4       Michael Penix  QB   76  100%    0    0%    0   0%
5         Ryan Neuzil  OL   76  100%    0    0%    0   0%
6    Casey Washington  WR   72   95%    0    0%    0   0%
7        Drake London  WR   67   88%    0    0%    0   0%
8      Bijan Robinson  RB   63   83%    0    0%    0   0%
9          Kyle Pitts  TE   59   78%    0    0%    0   0%
10    Ray-Ray McCloud  WR   55   72%    0    0%    1   4%
11    Charlie Woerner  TE   32   42%    0    0%   13  48%
12     Tyler Allgeier  RB   19   25%    0    0%    8  30%
13     KhaDarel Hodge  WR    8   11%    0    0%   17  63%
14  Teagan Quitoriano  TE    2    3%    0    0%   13  48%
15        David Sills  WR    2    3%    0    0%    0   0%
16     Feleipe

/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/2020286283.py:52: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(comment))[0]


In [23]:
from pdf2image import convert_from_path
import pytesseract

# Path to your GL balance report
pdf_path = "/users/coop/desktop/Financial Statements/Trial Balance July 2025.pdf"

# Convert PDF pages to images
pages = convert_from_path(pdf_path, dpi=300)

# OCR each page and print text
for i, page in enumerate(pages, start=1):
    text = pytesseract.image_to_string(page, lang="eng")
    print(f"\n--- PAGE {i} ---\n")
    print(text)


--- PAGE 1 ---

ASSETS,

102.
103.
107.
110.
-00
113.
114.
i258.
150.
155 .
160.
161.
162.
163.
164.
165.
168.
169.
170.
172.
173.5
180.
201.
202.
203.
210.
Zin
215)
216.
217.
218.
219i.
229.
23:0).
231.
231.
231.
305.
310.
BLD,
313.
315.

Liat

00
00
00
00

00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
10
20
00
00
00
00
00

INCOME

500.
525.
530.
-00

531

scan
535 .
53°74
540.

00
00
00

00
00
00
00

Whitehead Electric Company,

LIABILITIES & EQUITY

TOTAL

& EXPENSES

August 6, 2025

OPER. ACCT-CRAFT BANK
PAYROLL ACCT-CRAFT BANK
MONEY MARKET-CRAFT BANK
ACCOUNTS RECEIVABLE
ACCTS. REC. - RETAINAGE
COST IN EXCESS OF BILLIN
SHAREHOLDER LOAN

JOBS IN PROGRESS-MATERIA
INVENTORY

ROU - ASSET

FURN, FIXTURES & EQUIP.
DEPR .ALLOW- FURN&FXTRS
AUTO, TRUCK/TRAILERS
DEPR.ALLOW-AUTO, TRUCK/TR
EQUIPMENT & HEAVY TOOLS
DEPR. ALLOW-EQUIP&TOOLS
COMPUTERS

DEPR. ALLOW - COMPUTERS
LEASEHOLD IMPROVEMENTS
COPY MACHINES
DEPR.ALLOW-COPY MACHINES
CASH SURRENDER VA

In [26]:
# pip install selenium webdriver-manager bs4 lxml pandas

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import urlparse
import os
import re
import pandas as pd
from bs4 import BeautifulSoup, Comment

# ---- config ----
BASE_DIR = "/users/coop/desktop/Fantasy/2024"
TABLE_IDS = [
    "player_offense",
    "player_defense",
    "passing_advanced",
    "rushing_advanced",
    "receiving_advanced",
    "defense_advanced",
    "home_snap_counts",
    "vis_snap_counts",
    "home_drives",
    "vis_drives",
    "pbp",
]

def extract_game_id_from_url(url: str) -> str:
    """
    Example:
      https://www.pro-football-reference.com/boxscores/202509070buf.htm#home_snap_counts
      -> 202509070buf
    """
    path = urlparse(url).path  # /boxscores/202509070buf.htm
    fname = os.path.basename(path)  # 202509070buf.htm
    return re.sub(r"\.htm.*$", "", fname)

def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)
    return df

def parse_table_from_container(soup: BeautifulSoup, table_id: str) -> pd.DataFrame | None:
    """
    PFR often wraps tables in HTML comments inside a div#all_{table_id}.
    Try:
      1) Direct table inside #all_{table_id}
      2) First HTML comment within the container that holds a <table>
      3) Fallback: scan all comments on page (rarely needed)
    """
    container = soup.find(id=f"all_{table_id}")
    if container:
        # Case 1: direct table present
        direct = container.find("table")
        if direct is not None:
            tables = pd.read_html(str(direct))
            if tables:
                return flatten_headers(tables[0])

        # Case 2: commented table inside container
        comments = [n for n in container.descendants if isinstance(n, Comment)]
        for c in comments:
            if "<table" in c:
                tables = pd.read_html(str(c))
                if tables:
                    return flatten_headers(tables[0])

    # Case 3: fallback scan (should rarely be needed, but keeps it robust)
    comments = soup.find_all(string=lambda x: isinstance(x, Comment))
    for c in comments:
        if f'id="{table_id}"' in c or f'id="{table_id}"'.replace('"', "'") in c or table_id in c:
            if "<table" in c:
                tables = pd.read_html(str(c))
                if tables:
                    return flatten_headers(tables[0])
    return None

def save_tables_for_boxscore(url: str, base_dir: str = BASE_DIR) -> None:
    game_id = extract_game_id_from_url(url)

    # --- Selenium one-shot page load ---
    opts = Options()
    opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    try:
        driver.get(url)
        # Wait for the page shell to load; any always-present element near content works.
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div#content"))
        )
        html = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    # --- Extract and save each table ---
    saved, missing = [], []
    for table_id in TABLE_IDS:
        df = parse_table_from_container(soup, table_id)
        if df is None or df.empty:
            missing.append(table_id)
            continue

        out_dir = os.path.join(base_dir, table_id)  # table_name without '#'
        os.makedirs(out_dir, exist_ok=True)
        out_path = os.path.join(out_dir, f"{game_id}.csv")

        # Optional: ensure consistent string dtypes for safety
        # df = df.convert_dtypes()

        df.to_csv(out_path, index=False)
        saved.append(out_path)

    print(f"Game: {game_id}")
    print(f"Saved ({len(saved)}):")
    for p in saved:
        print("  -", p)
    if missing:
        print(f"Missing/empty ({len(missing)}): {', '.join(missing)}")

# ---- usage ----
# Replace with the single boxscore URL you provide:
# save_tables_for_boxscore("https://www.pro-football-reference.com/boxscores/202509070buf.htm#home_snap_counts")

In [27]:
save_tables_for_boxscore("https://www.pro-football-reference.com/boxscores/202509070buf.htm")

Game: 202509070buf
Saved (11):
  - /users/coop/desktop/Fantasy/2024/player_offense/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/player_defense/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/passing_advanced/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/rushing_advanced/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/receiving_advanced/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/defense_advanced/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/home_snap_counts/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/vis_snap_counts/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/home_drives/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/vis_drives/202509070buf.csv
  - /users/coop/desktop/Fantasy/2024/pbp/202509070buf.csv


/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/2294009599.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/2294009599.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/2294009599.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/2294009599.py:61: FutureWarning: Passing literal html to 'read_html'

In [48]:
# pip install selenium webdriver-manager bs4 lxml pandas

import os, re, time
from collections import deque
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup, Comment

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

BASE_DIR = "/users/coop/desktop/Fantasy/2021/files"
TABLE_IDS = [
    "player_offense","player_defense",
    "passing_advanced","rushing_advanced","receiving_advanced","defense_advanced",
    "home_snap_counts","vis_snap_counts","home_drives","vis_drives","pbp",
]

def flatten_headers(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)
    return df

def parse_table_from_container(soup, table_id):
    container = soup.find(id=f"all_{table_id}")
    if container:
        direct = container.find("table")
        if direct is not None:
            t = pd.read_html(str(direct))
            if t: return flatten_headers(t[0])
        comments = [n for n in container.descendants if isinstance(n, Comment)]
        for c in comments:
            if "<table" in c:
                t = pd.read_html(str(c))
                if t: return flatten_headers(t[0])
    # rare fallback
    for c in soup.find_all(string=lambda x: isinstance(x, Comment)):
        if table_id in c and "<table" in c:
            t = pd.read_html(str(c))
            if t: return flatten_headers(t[0])
    return None

def save_tables_from_html(game_id, html):
    soup = BeautifulSoup(html, "html.parser")
    for table_id in TABLE_IDS:
        df = parse_table_from_container(soup, table_id)
        out_dir = os.path.join(BASE_DIR, table_id)
        os.makedirs(out_dir, exist_ok=True)
        out_path = os.path.join(out_dir, f"{game_id}.csv")
        if df is None or df.empty:
            # still write an empty placeholder for traceability
            pd.DataFrame().to_csv(out_path, index=False)
            continue
        df.to_csv(out_path, index=False)

def throttle_factory(max_req=5, window_sec=60):
    hits = deque()
    def throttle():
        now = time.time()
        while hits and now - hits[0] > window_sec:
            hits.popleft()
        if len(hits) >= max_req:
            sleep_s = window_sec - (now - hits[0]) + 0.05
            if sleep_s > 0:
                time.sleep(sleep_s)
        hits.append(time.time())
    return throttle

def run_batch(game_ids):
    opts = Options()
    opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    throttle = throttle_factory(max_req=5, window_sec=60)

    try:
        for gid in game_ids:
            url = f"https://www.pro-football-reference.com/boxscores/{gid}.htm"
            throttle()  # ensure ≤5 page loads in any rolling 60s
            try:
                driver.get(url)
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div#content"))
                )
                html = driver.page_source
                save_tables_from_html(gid, html)
                print(f"OK {gid}")
            except Exception as e:
                print(f"FAIL {gid}: {e}")
    finally:
        driver.quit()

# ---- load ids and run ----
ids_df = pd.read_csv("/users/coop/desktop/Fantasy/2021/rep.csv", dtype=str)
game_ids = ids_df["home_id"].dropna().astype(str).str.strip().tolist()
run_batch(game_ids)

/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated 

OK 202109120was


/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated 

OK 202110030atl


/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated 

...

OK 202111290was


/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated 

OK 202112210phi


/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(str(direct))
/var/folders/q6/_cxy49v95vl9cwyck72pzgn00000gn/T/ipykernel_54551/4048913441.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated 

OK 202112260dal


In [47]:
import os

base_path = "/users/coop/desktop/Fantasy/2021/files"

# substrings to match in filenames (lowercase for case-insensitive match)
bad_strings = ["na", "202110030atl", "202111210car", "202112050rai", "202201090nyg", "202109260buf",
               "202110240gnb", "202110310den", "202112210phi", "202112260dal"]

for root, _, files in os.walk(base_path):
    for file in files:
        if file.lower().endswith(".csv"):
            if any(bad in file.lower() for bad in bad_strings):
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Removed: {file_path}")
                except Exception as e:
                    print(f"Error removing {file_path}: {e}")

Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202111140na.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202111210car.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202110310den.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202110100na.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202112210phi.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202110240gnb.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202110030atl.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202109260buf.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202112050rai.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202110170na.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202109120na.csv
Removed: /users/coop/desktop/Fantasy/2021/files/vis_snap_counts/202111290na.csv
Removed: /users/coop/desktop/Fant

In [49]:
import os
import pandas as pd

base_path = "/users/coop/desktop/Fantasy/2021/files"
rollup_path = "/users/coop/desktop/Fantasy/2021/rollups"
os.makedirs(rollup_path, exist_ok=True)

# get all first-level subfolders (the 11 folders)
folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    dfs = []
    
    for file in os.listdir(folder_path):
        if file.lower().endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path)
                # add filename (without .csv) as tag
                df["source_file"] = os.path.splitext(file)[0]
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    
    if dfs:
        rollup_df = pd.concat(dfs, ignore_index=True)
        out_file = os.path.join(rollup_path, f"{folder}_rollup.csv")
        rollup_df.to_csv(out_file, index=False)
        print(f"Saved rollup: {out_file} ({len(rollup_df)} rows)")
    else:
        print(f"No CSVs found in {folder_path}")

Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/vis_snap_counts_rollup.csv (14709 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/home_snap_counts_rollup.csv (14738 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/player_defense_rollup.csv (11623 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/home_drives_rollup.csv (2929 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/player_offense_rollup.csv (5980 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/passing_advanced_rollup.csv (947 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/rushing_advanced_rollup.csv (2546 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/pbp_rollup.csv (48865 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/receiving_advanced_rollup.csv (4681 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/defense_advanced_rollup.csv (8343 rows)
Saved rollup: /users/coop/desktop/Fantasy/2021/rollups/vis_drives_ro

In [42]:
# pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup

def print_html(url: str, timeout: int = 15):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/123.0 Safari/537.36"
    }
    r = requests.get(url, headers=headers, timeout=timeout)
    r.raise_for_status()
    r.encoding = r.apparent_encoding  # better decoding for some sites
    soup = BeautifulSoup(r.text, "html.parser")  # or "lxml" if installed
    # Pretty-printed HTML
    print(soup.prettify())

# example:
# print_html("https://example.com")